# 先验算法
先验算法（Apriori Algorithm是关联规则学习的经典算法之一。先验算法的设计目的是为了处理包含交易信息内容的数据库（例如,顾客购买的商品清单，或者网页常访清单。）而其他的算法则是设计用来寻找无交易信息（如Winepi算法和Minepi算法）或无时间标记（如DNA测序）的数据之间的联系规则。

在关联式规则中,一般对于给定的项目集合（例如，零售交易集合，每个集合都列出的单个商品的购买信息），算法通常尝试在项目集合中找出至少有C个相同的子集。先验算法采用自底向上的处理方法，即频繁子集每次只扩展一个对象（该步骤被称为候选集产生），并且候选集由数据进行检验。当不再产生匹配条件的扩展对象时，算法终止
<img src="aprior.png" style="width:400px;height:300px;">
先验算法采用广度优先搜索算法进行搜索并采用树结构来对候选项目集进行高效计数。它通过长度为 ${\displaystyle k-1}$的候选项目集来产生长度为$ {\displaystyle k} $的候选项目集，然后从中删除包含不常见子模式的候选项。根据向下封闭性引理,该候选项目集包含所有长度为 ${\displaystyle k}$的频繁项目集。之后，就可以通过扫描交易数据库来决定候选项目集中的频繁项目集。
## Apriori 定律
- 如果一个集合是频繁项集，则它的所有子集都是频繁项集。举例：假设一个集合{A,B}是频繁项集，即A、B同时出现在一条记 录的次数大于等于最小支持度min_support，则它的子集{A},{B}出现次数必定大于等于min_support，即它的子集都是频繁项集。

- 如果一个集合不是频繁项集，则它的所有超集都不是频繁项集。举例：假设集合{A}不是频繁项集，即A出现的次数小于min_support，则它的任何超集如{A,B}出现的次数必定小于min_support，因此其超集必定也不是频繁项集。

## 代码实现
**辅助函数**

In [1]:
from numpy import *

def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [2]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we

In [19]:
def scanD(D, Ck, minSupport):
    # 相当于遍历D，求出所有的Ck中的元素支持度大于minsupport的
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0)+1
#                 if can not in ssCnt: ssCnt[can]=1
#                 else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

函数`loadDataSet()`创建了一个用于测试的简单数据集.函数`createC1()`将构建集合C1。C1是大小为1的所有候选项集的集合。Apriori算法首先构建集合C1 ,然后扫描数据集来判断这些只有一个元素的项集是否满足最小支持度的要求。那些满足最低要求的项集构成集合L1。而L1中的元素相互组合构成C2，C2再进一步过滤变为L2。

函数`createC1()`来构建第一个候选项集的列表C1。由于算法一开始是从输人数据中提取候选项集列表，所以这里需要一个特殊的函数来处理，而后续的项集列表则是按一定的格式存放的。这里使用的格式就是Python中的frozenset类型。frozenset是指被“冰冻”的集合，就是说它们是不可改变的，即用户不能修改它们。这里必须要使用frozenset而不是set类型，因为之后必须要将这些集合作为字典键值使用，使用frozenset可以实现这一点，而set却做不到。(字典，集合和列表都是可变的，不能当作字典的键)

首先创建一个空列表C1 ,它用来存储所有不重复的项值。接下来遍历数据集中的所有交易记录。对每一条记录，遍历记录中的每一个项。如果某个物品项没有在C1中出现,则将其添加到C1中。这里并不是简单地添加每个物品项，而是添加只包含该物品项的一个列表。这样做的目的是为每个物品项构建一个集合。因为在Apriori算法的后续处理中，需要做集合操作。Python不能创建只有一个整数的集合，因此这里实现必须使用列表（有兴趣的读者可以试一下)。这就是我们使用一个由单物品列表组成的大列表的原因。最后，对大列表进行排序并将其中的每个单元素列表映射到frozenset()，最后返回frozenset的列表

函数是`scanD()`有三个参数,分别是数据集Ck包含候选集合的列表以及感兴趣项集的最小支持度miniSupport。该函数用于从C1生成C2。另外，该函数会返回一个包含支持度值的字典以备后用.`scanD()`函数首先创建一个空字典ssCnt，然后遍历数据集中的所有交易记录以及01中的所有候选集。如果01中的集合是记录的一部分，那么增加字典中对应的计数值。这里字典的键就是集合。当扫描完数据集中的所有项以及所有候选集时，就需要计算支持度。不满足最小支持度要求的集合不会输出。函数也会先构建个空列表，该列表包含满足最小支持度要求的集合。下一个循环遍历字典中的每个元素并且计算支持度。如果支持度满足最小支持度要求，则将字典元素添加到retList中。可以使用语句retList.insert(0, key)在列表的首部插人任意新的集合。当然也不一定非要在首部插入，这只是为了让列表看起来有组织。函数最后返回最频繁项集的支持度supportData

In [20]:
# test
dataSet = loadDataSet()
print(dataSet)
C1 = createC1(dataSet)
print(C1)
D = list(map(set,dataSet))
print(D)
L1, supportData0 = scanD(D, C1, 0.5)
print(L1)


[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]
[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]
